<style>
body {
    font-size: 20pt !important;
}

.rendered_html {
    font-size: 20pt !important;
}

.CodeMirror pre {
    font-size: 20pt !important;
}

.output pre {
    font-size: 20pt !important;
}
</style>


<h2 style="font-size:24pt"> Proyecto DESI</h2>

<h2 style="font-size:24pt"> Julio 3, 2025</h2>

<p style="font-size:16pt">
Conversión de datos desde RAC, DEC y z a coordenadas esféricas y finalmente a coordenadas x,y y z.
</p>


## Data

In [104]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.cosmology import Planck18
from astropy.io import ascii
import random

In [5]:
# original files are found here https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/
data_n = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_clustering.dat.fits")
data_s = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_clustering.dat.fits")

In [100]:
list_rand = list(range(18))
rand_tables_n = {}

for i in list_rand:
    url = f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_{i}_clustering.ran.fits"
    rand_tables_n[f"rand_{i}_n"] = Table.read(url)

In [163]:
rand_tables_s = {}

for i in list_rand:
    url = f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_{i}_clustering.ran.fits"
    rand_tables_s[f"rand_{i}_s"] = Table.read(url)

<p style="font-size:16pt">
$$\theta = 90 - \delta$$
$$\varphi = -\alpha$$
$$ z = r [Mpc] $$

In [20]:
def esfericas(data):
    data['theta']= 90 - data['DEC']
    data['phi'] = - data['RA']
    data['r'] = Planck18.comoving_distance(data['Z'])

In [21]:
esfericas(data_n)
esfericas(data_s)

<p style="font-size:16pt">
\begin{align*}
x &= r \sin\theta \cos\varphi \\
y &= r \sin\theta \sin\varphi \\
z &= r \cos\theta
\end{align*}


In [22]:
def cartesianas(data):
    data['x'] = data['r']*np.sin(data['theta'])*np.cos(data['phi'])
    data['y'] = data['r']*np.sin(data['theta'])*np.sin(data['phi'])
    data['z'] = data['r']*np.cos(data['theta'])

In [23]:
cartesianas(data_n)
cartesianas(data_s)

In [165]:
rand_tables_n.keys()

dict_keys(['rand_0_n', 'rand_1_n', 'rand_2_n', 'rand_3_n', 'rand_4_n', 'rand_5_n', 'rand_6_n', 'rand_7_n', 'rand_8_n', 'rand_9_n', 'rand_10_n', 'rand_11_n', 'rand_12_n', 'rand_13_n', 'rand_14_n', 'rand_15_n', 'rand_16_n', 'rand_17_n'])

In [166]:
rand_tables_s.keys()

dict_keys(['rand_0_s', 'rand_1_s', 'rand_2_s', 'rand_3_s', 'rand_4_s', 'rand_5_s', 'rand_6_s', 'rand_7_s', 'rand_8_s', 'rand_9_s', 'rand_10_s', 'rand_11_s', 'rand_12_s', 'rand_13_s', 'rand_14_s', 'rand_15_s', 'rand_16_s', 'rand_17_s'])

In [194]:
def save_file(number_rosette,data,orientation):

    if orientation == 'n':
        keys = rand_tables_n
    elif orientation == 's':
        keys = rand_tables_s
    
    #Choose random table
    clave_aleatoria = random.choice(list(keys.keys()))
    rand_table = keys[clave_aleatoria]

    #Filter of the rosette
    rosette = data[data['ROSETTE_NUMBER']==number_rosette]
    rand_table_rosette = rand_table[rand_table['ROSETTE_NUMBER']==number_rosette]
        
    #Choose random index of the random table according to the len of rosette table
    len_rosette = len(rosette)
    idx_random = np.random.choice(len(rand_table_rosette), size=len_rosette, replace=False)
    rand_subset = rand_table_rosette[idx_random]

    #Add columns of the othe coordinates
    esfericas(rand_subset)
    cartesianas(rand_subset)

    #Filenames
    filename_data = f'data_rosette/LRG_{number_rosette}_clustering_data.ecsv'
    filename_rand = f'data_rosette/LRG_{number_rosette}_clustering_rand.ecsv'

    #Write
    ascii.write(rosette, filename_data, format='ecsv', overwrite=True)
    ascii.write(rand_subset, filename_rand, format='ecsv', overwrite=True)

In [195]:
#Save files north
for i in set(data_n['ROSETTE_NUMBER']):
    save_file(i,data_n,'n')

In [196]:
set(data_n['ROSETTE_NUMBER'])

{3, 6, 7, 11, 12, 13, 14, 15, 18, 19}

In [197]:
#Save files south
for i in set(data_s['ROSETTE_NUMBER']):
    save_file(i,data_s,'s')

In [198]:
set(data_s['ROSETTE_NUMBER'])

{0, 1, 2, 4, 5, 8, 9, 10, 16, 17}